In [1]:
from subspace_partition.subspace_partition import (
    run_subspace_partition,
    SubspacePartitionConfig,
)

import transformer_lens
from pathlib import Path
import copy_transformer.data
import copy_transformer.tokenizer

In [2]:
EMBEDDING_DIM = 64
NUM_HEADS = 8
VOCABULARY = [c for c in "ABCDEFGHIJKLMNOPQRSTUVWXYZ"]
CONTEXT_LENGTH = 32

MAX_PATTERN_LENGTH = 16

MAX_STEPS = 10_000

EXPERIMENT_NAME = "copy_transformer"
OUTPUT_DIR = Path("out/subspace_partition")

In [3]:
tokenizer = copy_transformer.tokenizer.SingleCharTokenizer(
    alphabet=VOCABULARY,
    bos_token=">",
    eos_token="<",
    unk_token="?",
    pad_token="_",
    name_or_path="custom",
    add_bos_token=True,
)

model_config = transformer_lens.HookedTransformerConfig(
    d_model=EMBEDDING_DIM,
    d_head=EMBEDDING_DIM // NUM_HEADS,
    n_layers=2,
    n_ctx=CONTEXT_LENGTH,
    n_heads=NUM_HEADS,
    d_vocab=tokenizer.vocab_size,
    attn_only=True,
)
model_state_dict_path = Path("out/copy_transformer.pt")

dataset = copy_transformer.data.InfinitePureRepeatingPatternDataset(
    vocabulary=VOCABULARY,
    context_length=CONTEXT_LENGTH,
    max_pattern_length=MAX_PATTERN_LENGTH,
)

subspace_partition_config = SubspacePartitionConfig(
    exp_name=EXPERIMENT_NAME,
    output_dir=OUTPUT_DIR,
    model_config=model_config,
    model_weights_path=model_state_dict_path,
    act_sites=["blocks.0.hook_resid_post", "blocks.1.hook_resid_post"],
    tokenizer=tokenizer,
    dataset=dataset,
    max_steps=MAX_STEPS,
)

In [4]:
run_subspace_partition(subspace_partition_config)

training for blocks.0.hook_resid_post


/home/juliusk/_/Uni/Kurse_Winter_2025_2026/AI Safety Incubator/Research_Project/SubspacePartition/.venv/lib/python3.12/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
  2%|▏         | 201/10000 [00:30<25:19,  6.45it/s]

{'R_grad_norm': 0.018802380366250874, 'training_loss': 0.19663480170071124}


  4%|▍         | 401/10000 [01:00<23:45,  6.73it/s]

{'R_grad_norm': 0.021874440442770718, 'training_loss': 0.1862760491669178}


...

{'R_grad_norm': 0.8883127182722091, 'training_loss': 1.7186156529188157}


 98%|█████████▊| 9801/10000 [25:28<00:29,  6.67it/s]

{'R_grad_norm': 0.8799321511387825, 'training_loss': 1.7148295032978058}


100%|██████████| 10000/10000 [25:58<00:00,  6.42it/s]


{'R_grad_norm': 0.8736403515934944, 'training_loss': 1.711547617316246}
finish training (10000)
saving.. out/subspace_partition/copy_transformer
evaluating (25 steps)...
 ******* eval result *******
mean (weighted) 1.7082897424697876
mean (unweighted) 1.7082897424697876
tensor([1.4647, 1.9519])
